Right now, I plan to follow along with this tutorial for utilizing PVlib: [PyData 2021: Solar PV Modeling](https://pvsc-python-tutorials.github.io/pyData-2021-Solar-PV-Modeling/Tutorial%201%20-%20TMY%20Weather%20Data.html)

- The location I am interested in forecasting for is: 39.043, -94.681
- Not sure how PVlib works yet, but I'd like to model a residential deployment.  Here are some factors I want to work with:
  - South Facing roof
  - 10/12 pitch (39.81 degrees)
  - Some shading on the eastern side below 20 degrees
  - Some shading on the southern side below 20 degrees (may be closer to 15)
  - Some shading on western side below 15 degrees

### Atmospheric Data

TMY: Typical Meteorological Year at a location.  Contains:
- Solar Irradiance
- Air Temperature (influences PV performance)
- Wind Speed

This tutorial seems to use a standard file for TMY, but that was last updated in 2015 and I'm not sure what locations I could obtain.  Instead, I'll use the PVGIS data via the get_pvgis_tmy function.

In [2]:
import os
import pandas as pd
import pvlib

import matplotlib.pyplot as plt

In [5]:
lat = 39.043
lon = -94.681

In [4]:
help(pvlib.iotools.get_pvgis_tmy)

Help on function get_pvgis_tmy in module pvlib.iotools.pvgis:

get_pvgis_tmy(latitude, longitude, outputformat='json', usehorizon=True, userhorizon=None, startyear=None, endyear=None, url='https://re.jrc.ec.europa.eu/api/', map_variables=None, timeout=30)
    Get TMY data from PVGIS.
    
    For more information see the PVGIS [1]_ TMY tool documentation [2]_.
    
    Parameters
    ----------
    latitude : float
        Latitude in degrees north
    longitude : float
        Longitude in degrees east
    outputformat : str, default 'json'
        Must be in ``['csv', 'basic', 'epw', 'json']``. See PVGIS TMY tool
        documentation [2]_ for more info.
    usehorizon : bool, default True
        include effects of horizon
    userhorizon : list of float, default None
        optional user specified elevation of horizon in degrees, at equally
        spaced azimuth clockwise from north, only valid if ``usehorizon`` is
        true, if ``usehorizon`` is true but ``userhorizon`` is ``

Notes:
- This function uses all years available via PVGIS as opposed to PVGIS's default of 10 years.  So I'm going to specify the last 20 years fo data.
  - Whelp, that didn't work.  pvlib seems to use a pvgis api that only supports data from 2005 to 2015.  So it would seem like I have no option on what 10 years of data to use
  - That being said, [PVGIS documentation](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system_en) indicates that v5.2 contains data through 2020.  DECISION: Use default for now, investigate directly using the PVGIS API in the future for TMY based on more recent data.


In [9]:
pvg = pvlib.iotools.get_pvgis_tmy(lat,lon,map_variables=True)

In [10]:
type(pvg)

tuple

NOTE: Tuple should be **(TMY df, list of months selected, dictionary of inputs, metadata)**


In [18]:
tmy = pvg[0] #dataframe

In [19]:
tmy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2005-01-01 00:00:00+00:00 to 2014-12-31 23:00:00+00:00
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   temp_air           8760 non-null   float64
 1   relative_humidity  8760 non-null   float64
 2   ghi                8760 non-null   float64
 3   dni                8760 non-null   float64
 4   dhi                8760 non-null   float64
 5   IR(h)              8760 non-null   float64
 6   wind_speed         8760 non-null   float64
 7   wind_direction     8760 non-null   float64
 8   pressure           8760 non-null   float64
dtypes: float64(9)
memory usage: 684.4 KB


In [17]:
#check out the month's that were selected
pvg[1]

[{'month': 1, 'year': 2005},
 {'month': 2, 'year': 2006},
 {'month': 3, 'year': 2009},
 {'month': 4, 'year': 2009},
 {'month': 5, 'year': 2014},
 {'month': 6, 'year': 2006},
 {'month': 7, 'year': 2011},
 {'month': 8, 'year': 2012},
 {'month': 9, 'year': 2013},
 {'month': 10, 'year': 2014},
 {'month': 11, 'year': 2005},
 {'month': 12, 'year': 2014}]

In [20]:
#Check out the inputs
pvg[2]

{'location': {'latitude': 39.043, 'longitude': -94.681, 'elevation': 294.0},
 'meteo_data': {'radiation_db': 'PVGIS-NSRDB',
  'meteo_db': 'ERA-Interim',
  'year_min': 2005,
  'year_max': 2015,
  'use_horizon': True,
  'horizon_db': 'DEM-calculated'}}

In [21]:
meta = pvg[3]
meta

{'inputs': {'location': {'description': 'Selected location',
   'variables': {'latitude': {'description': 'Latitude',
     'units': 'decimal degree'},
    'longitude': {'description': 'Longitude', 'units': 'decimal degree'},
    'elevation': {'description': 'Elevation', 'units': 'm'}}},
  'meteo_data': {'description': 'Sources of meteorological data',
   'variables': {'radiation_db': {'description': 'Solar radiation database'},
    'meteo_db': {'description': 'Database used for meteorological variables other than solar radiation'},
    'year_min': {'description': 'First year of the calculations'},
    'year_max': {'description': 'Last year of the calculations'},
    'use_horizon': {'description': 'Include horizon shadows'},
    'horizon_db': {'description': 'Source of horizon data'}}}},
 'outputs': {'months_selected': {'type': 'time series',
   'timestamp': 'monthly',
   'description': 'months selected for the TMY'},
  'tmy_hourly': {'type': 'time series',
   'timestamp': 'hourly',
   